# 要素名称

In [2]:
import xarray as xr

xr.set_options(display_style="text")

nwpc-data 中的 `load_` 系列函数，使用 `parameter` 参数指定想要获取的要素名称。

`parameter` 可以接收以下类型的参数：

- `str`：使用变量的 `shortName` 检索
- `dict`：设置用于过滤的键值对

In [3]:
from reki.format.grib.eccodes import (
    load_message_from_file,
    load_field_from_file,
)

## 准备数据

本示例使用 GRAPES GFS 的 GRIB 2 数据为例。

获取 GRAPES GFS 2020 年 4 月 19 日 00 时次 024 时效的 GRIB 2 文件路径

In [4]:
from reki.data_finder import find_local_file

data_path = find_local_file(
    "grapes_gfs_gmf/grib2/orig",
    start_time="2020041900",
    forecast_time="024h",
)
data_path

PosixPath('/sstorage1/COMMONDATA/OPER/NWPC/GRAPES_GFS_GMF/Prod-grib/2020041900/ORIG/gmf.gra.2020041900024.grb2')

## shortName

通常情况下，可以使用要素的 `shortName`，比如温度 `t`，位势高度 `gh`，湿度 `q` 等等。

In [5]:
t850 = load_field_from_file(
    data_path,
    parameter="t",
    level_type="isobaricInhPa",
    level=850,
)
t850

<xarray.DataArray 't' (latitude: 720, longitude: 1440)>
array([[265.05535156, 265.09535156, 265.05535156, ..., 265.11535156,
        265.04535156, 265.08535156],
       [264.94535156, 264.94535156, 264.97535156, ..., 264.94535156,
        264.92535156, 264.94535156],
       [264.76535156, 264.77535156, 264.79535156, ..., 264.75535156,
        264.74535156, 264.77535156],
       ...,
       [242.90535156, 242.60535156, 242.67535156, ..., 242.81535156,
        242.77535156, 242.77535156],
       [242.60535156, 242.61535156, 242.61535156, ..., 242.55535156,
        242.71535156, 242.77535156],
       [242.55535156, 242.53535156, 242.63535156, ..., 242.57535156,
        242.53535156, 242.57535156]])
Coordinates:
    time           datetime64[ns] 2020-04-19
    step           timedelta64[ns] 1 days
    valid_time     datetime64[ns] 2020-04-20
    isobaricInhPa  int64 850
  * latitude       (latitude) float64 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * longitude      (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
Attributes:
    GRIB_edition:                    2
    GRIB_centre:                     babj
    GRIB_subCentre:                  0
    GRIB_tablesVersion:              4
    GRIB_localTablesVersion:         1
    GRIB_dataType:                   fc
    GRIB_dataDate:                   20200419
    GRIB_dataTime:                   0
    GRIB_validityDate:               20200420
    GRIB_validityTime:               0
    GRIB_step:                       24
    GRIB_stepType:                   instant
    GRIB_stepUnits:                  1
    GRIB_stepRange:                  24
    GRIB_endStep:                    24
    GRIB_name:                       Temperature
    GRIB_shortName:                  t
    GRIB_cfName:                     air_temperature
    GRIB_discipline:                 0
    GRIB_parameterCategory:          0
    GRIB_parameterNumber:            0
    GRIB_gridType:                   regular_ll
    GRIB_gridDefinitionDescription:  Latitude/longitude 
    GRIB_typeOfFirstFixedSurface:    pl
    GRIB_typeOfLevel:                isobaricInhPa
    GRIB_level:                      850
    GRIB_numberOfPoints:             1036800
    GRIB_missingValue:               9999
    GRIB_units:                      K
    long_name:                       Temperature
    units:                           K

`shortName` 是 eccodes 中使用的 GRIB key。
具体名称可以运行 `grib_ls` 命令浏览 GRIB 2 文件。

下面的命令使用 `grib_ls` 浏览 GRIB 2 文件，只显示前 10 行输出结果。

In [6]:
!grib_ls {data_path} | head -n 10

/sstorage1/COMMONDATA/OPER/NWPC/GRAPES_GFS_GMF/Prod-grib/2020041900/ORIG/gmf.gra.2020041900024.grb2
edition      centre       date         dataType     gridType     typeOfLevel  level        stepRange    shortName    packingType  
2            babj         20200419     fc           regular_ll   surface      0            0-24         acpcp        grid_jpeg   
2            babj         20200419     fc           regular_ll   surface      0            0-24         ncpcp        grid_jpeg   
2            babj         20200419     fc           regular_ll   surface      0            0-24         unknown      grid_jpeg   
2            babj         20200419     fc           regular_ll   surface      0            0-24         asnow        grid_jpeg   
2            babj         20200419     fc           regular_ll   surface      0            24           t            grid_jpeg   
2            babj         20200419     fc           regular_ll   surface      0            0-24         str          gr

可以看到，`grib_ls` 仅能识别部分要素场的 `shortName`，无法识别的被标记为 `unknown`。

对于这类变量，就需要使用到下面的方法。

### 练习

检索 500hPa 位势高度场（`gh`）

## dict

GRIB 2 消息的变量可以通过三个 GRIB Key 唯一确定：

- discipline
- parameterCategory
- parameterNumber

`parameter` 参数支持 dict 类型，用于指定包含上述 key 的键值对形式的过滤条件。

我们首先使用 `grib_ls` 命令看一下前三个要素场的 key 值。

In [7]:
!grib_ls -w count=1/2/3 -p discipline,parameterCategory,parameterNumber,shortName,name {data_path}

/sstorage1/COMMONDATA/OPER/NWPC/GRAPES_GFS_GMF/Prod-grib/2020041900/ORIG/gmf.gra.2020041900024.grb2
discipline         parameterCategory  parameterNumber    shortName          name               
0                  1                  10                 acpcp              Convective precipitation (water) 
0                  1                  9                  ncpcp              Large scale precipitation (non-convective) 
0                  1                  8                  unknown            unknown           
3 of 3 messages in /sstorage1/COMMONDATA/OPER/NWPC/GRAPES_GFS_GMF/Prod-grib/2020041900/ORIG/gmf.gra.2020041900024.grb2

3 of 3 total messages in 1 files


可以使用下面的代码检索第三个要素场

In [8]:
apcp = load_field_from_file(
    data_path,
    parameter={
        "discipline": 0,
        "parameterCategory": 1,
        "parameterNumber": 8
    },
    level_type="surface",
    level=0,
)
apcp

<xarray.DataArray 'unknown' (latitude: 720, longitude: 1440)>
array([[0.57648 , 0.576544, 0.57656 , ..., 0.576176, 0.576224, 0.576416],
       [0.542784, 0.542736, 0.542816, ..., 0.542928, 0.542672, 0.54264 ],
       [0.524768, 0.52544 , 0.525712, ..., 0.523952, 0.524896, 0.52528 ],
       ...,
       [0.372256, 0.373584, 0.365424, ..., 0.36592 , 0.365648, 0.36568 ],
       [0.340464, 0.338672, 0.34344 , ..., 0.32928 , 0.335216, 0.337184],
       [0.311344, 0.309776, 0.30952 , ..., 0.305648, 0.311072, 0.309088]])
Coordinates:
    time        datetime64[ns] 2020-04-19
    step        timedelta64[ns] 1 days
    valid_time  datetime64[ns] 2020-04-20
    surface     int64 0
  * latitude    (latitude) float64 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
Attributes:
    GRIB_edition:                    2
    GRIB_centre:                     babj
    GRIB_subCentre:                  0
    GRIB_tablesVersion:              4
    GRIB_localTablesVersion:         1
    GRIB_dataType:                   fc
    GRIB_dataDate:                   20200419
    GRIB_dataTime:                   0
    GRIB_validityDate:               20200420
    GRIB_validityTime:               0
    GRIB_step:                       24
    GRIB_stepType:                   accum
    GRIB_stepUnits:                  1
    GRIB_stepRange:                  0-24
    GRIB_endStep:                    24
    GRIB_discipline:                 0
    GRIB_parameterCategory:          1
    GRIB_parameterNumber:            8
    GRIB_gridType:                   regular_ll
    GRIB_gridDefinitionDescription:  Latitude/longitude 
    GRIB_typeOfFirstFixedSurface:    sfc
    GRIB_typeOfLevel:                surface
    GRIB_level:                      0
    GRIB_numberOfPoints:             1036800
    GRIB_missingValue:               9999
    long_name:                       discipline=0 parmcat=1 parm=8

实际上，使用 `wgrib2` 命令查看可以得知，第三个要素场就是总降水 APCP。
在 GRIB 2 表格中的描述如下：

```
Total precipitation  (kg m-2)  (grib2/tables/4/4.2.0.1.table) 
```

### 练习

检索 850hPa 水汽通量。

```json
{
    "dicsipline": 0,
    "paramterCategory": 1,
    "paramterNumber": 224,
}
```

## 默认值

`parameter` 参数的默认值为 `None`，在查找消息时不检测要素场的变量。

与 `level_type` 和 `level` 的默认值一同使用，可以用于从文件中读取第一个要素场。当文件中只有一个要素场的时候非常有用。

**注意**：当前实现方式尚未优化，请谨慎使用默认参数读取多要素场文件，可能会耗费大量时间